## Inleiding

Hallo allemaal,
  
Welkom bij deze post over een tweede brein! Geen zorgen, het is geen brein op water in een potje en het is ook (nog) geen werkzaam brein. Het is een term die komt van Forte Labs (een bedrijf dat zich focust op het verbeteren van je creatieve geest) en simpel gezegd is het een systeem om efficiënt notities van al je geconsumeerde informatie te maken en op te slaan.  

Maar waarom wil je een tweede brein? Tegenwoordig is er zo veel informatie dat je het niet allemaal kan onthouden (eigenlijk was dat al een lange tijd zo en werden er boeken geschreven, maar met het internet heb je makkelijker toegang tot de vele informatie). Daarom zou het ideaal zijn om een systeem te gebruiken dat veel van de informatie voor jou onthoudt. Dan is het belangrijk om makkelijk bij de juiste informatie te komen, dat is namelijk mijn probleem wanneer ik al mijn notities opsla in schrijfblokken en schriften. Gelukkig hebben we nu hele mooie technische tools waarmee je gemakkelijk je notities kunt opzoeken! In deze post ga ik uitleggen welke technische tools ik gebruik en hoe ik een persoonlijke cloud heb gebouwd om al die data in eigen beheer te hebben (geen last van privacyschendingen). Laten we beginnen.

## Het Tweede Brein

Het belangrijkste aan het tweede brein is dat het makkelijk te gebruiken is. Het moet gemakkelijk zijn om snel notities te maken en alle notities moeten verzameld worden op 1 plek zodat je op elk apparaat (mobiel, laptop, desktop, e-reader, etc.) ten allen tijde bij al je notities kunt. En als je snel wilt beginnen moet het systeem makkelijk op te zetten zijn.

In alle eerlijkheid, de technieken die ik gebruik zijn verre van gemakkelijk op te zetten. Dat komt namelijk omdat ik de laatste tijd privacy belangrijker vind. Hierdoor vallen de technologieën die de data voor je beheren af omdat ik mijn data liever in eigen beheer heb. Als jij privacy minder belangrijk vindt, dan hoef je de manier in deze blogpost niet te gebruiken en kan je andere technologieën kiezen. Maar mocht je privacy wel degelijk zo belangrijk vinden (of je vindt het gewoon cool om zelf een opslag "cloud" te maken) dan hoop ik met deze blogpost het proces te vereenvoudigen!

De technologieën die ik heb gekozen voor mijn tweede brein zijn [Obsidian](https://obsidian.md) en [Nextcloud](https://nextcloud.com) (in plaats van Nextcloud kan je bijvoorbeeld Obsidian Sync of Evernote gebruiken, deze twee oplossingen beheren de data voor jou). Obsidian is een tool om notities te maken en Nextcloud is de tool om de data in op te slaan. Daarnaast zijn er nog wat tools nodig om de technologieën makkelijker te laten werken met bijvoorbeeld mobiele apparaten. Deze tools zijn [MacroDroid](https://www.macrodroid.com), [Syncthing](https://syncthing.net) en [Readwise](https://readwise.io) (ook de Reader). Deze laatste drie tools (Syncthing, Readwise en de Reader) zijn niet de focus van deze post, dus er komt nog een vervolg!  

Obsidian is relatief makkelijk in te stellen en te installeren. Helaas is Nextcloud wat ingewikkelder. Deze tool moet je namelijk ook beschikbaar maken via het internet (ik heb gekozen om alleen op mijn lokale netwerk bij de data te kunnen). Als eerste heb je een DNS Record nodig. Dan host je een reverse proxy zodat je via HTTPS kunt communiceren (Nginx). Vervolgens draai je Nextcloud op een server thuis (bijvoorbeeld zoals ik dat heb gedaan op een Raspberry Pi). De laatste stap is het configureren van de applicaties zodat je via je domein bij de Nextcloud app kan komen.

## DNS Record

Een DNS record is nodig om een domein (of subdomein) te laten wijzen naar je publieke IP adres (te vinden via https://www.whatismyip.com). Hierdoor kan je straks je Nextcloud server benaderen via het Wereldwijde Web. Nu heb ik zelf al een domein (waar je deze post op leest) en heb ik een subdomein gebruikt om naar het ip-adres te wijzen. Maar als je geen domein hebt, kan je ook een gratis subdomein gebruiken van bijvoorbeeld [DuckDNS](https://www.duckdns.org/about.jsp) (dat is gratis, zelf ben ik huiverig met gratis. Vaak doen ze dan iets met je data, ook al zeggen ze van niet. Dus voordat je DuckDNS gebruikt, onderzoek goed wat ze precies doen!). Vervolgens maak je een record (bijvoorbeeld een A (Alias) record) zoals hieronder in de afbeelding.


![](images/Pasted%20image%2020230513211440.png)


## Nginx Proxy

### Docker

Tegenwoordig is het draaien van applicaties zoals Nginx makkelijk gemaakt door het gebruik van Docker containers. Ik wil niet uitvoerig ingaan op wat een container is maar hier een quote van [Docker Website](https://www.docker.com/resources/what-container/)

>”A container is a standard unit of software that packages up code and all its dependencies so the application runs quickly and reliably from one computing environment to another”

Dus Docker moet geinstalleerd worden, en in [dit artikel](https://www.simplilearn.com/tutorials/docker-tutorial/raspberry-pi-docker) staat uitgelegd hoe je Docker kunt installeren op je Pi.

### Portainer

Vervolgens adviseer ik om [Portainer](https://www.portainer.io) te installeren. Portainer kan, net als Docker, zelf weer het beste uitleggen wat het is

>”Portainer is a universal container management platform that doesn't lock you into a single technology or vendor”

En het gemakkelijkste om Portainer te installeren is via Docker, en in [dit artikel](https://pimylifeup.com/raspberry-pi-portainer/) wordt uitgelegd hoe je dat doet.

Met Portainer en Docker geïnstalleerd kan er nu een Nginx stack gemaakt worden. Open de Portainer web interface, navigeer naar Stacks en klik op "Add Stacks". Dan geef je een naam aan de stack: bijvoorbeeld nginx-proxy. En in de Web Editor plak je onderstaande YAML code.

```yaml
version: "3"  
  
networks:  
  proxy:  
    external: true  
  
services:  
  reverse-proxy:  
    image: "jc21/nginx-proxy-manager:latest"  
    restart: always  
    ports:  
      - "80:80"  
      - "443:443"  
      - "81:81"  
    environment:  
      DB_SQLITE_FILE: "/data/database.sqlite"  
      DISABLE_IPV6: "true"  
    volumes:  
      - ./data:/data  
      - ./letsencrypt:/etc/letsencrypt  
    networks:  
      - proxy
```

Wanneer je je stack opslaat dan wordt er op de achtergrond [docker compose](https://docs.docker.com/compose/) gebruikt. Vaak wordt dit gebruikt voor multi-container Docker applicaties, maar ik vind het ook een overzichtelijke manier om een single-container applicatie te starten.

Hier een korte uitleg wat er wordt gemaakt in bovenstaande stack. Er wordt 1 service gestart, namelijk de reverse-proxy. Hierin wordt de nginx-proxy-manager docker image gebruikt. [Nginx Proxy Manager](https://nginxproxymanager.com/setup/) is een applicatie dat nginx en letsencrypt gebruikt om het zo makkelijk mogelijk te maken om een proxy server met HTTPS capaciteit op te zetten. Deze nginx-proxy wordt verbonden aan het netwerk "proxy" en dit netwerk is external. Dat is belangrijk omdat het netwerk dan ook benaderd kan worden in de Nextcloud stack. Voor meer informatie over de Nginx Proxy Manager [kan je hierheen gaan](https://nginxproxymanager.com/guide/#project-goal).

Navigeer naar de Nginx Proxy Manager (als je niks aangepast hebt aan bovenstaande config is dat http://[Lokale IP van Raspberry]:81). Als alles goed is gegaan komt er een inlog scherm waarin je kan inloggen met de volgende standaardwaarden

```
Email:    admin@example.com
Password: changeme
```

Nadat je inlogt met deze gegevens zal je onmiddellijk gevraagd worden om bovenstaande gegevens te wijzigen.

Nu je succesvol bent ingelogd is de volgende stap om de Nextcloud stack werkend te krijgen.


## Nextcloud

Om de Nextcloud applicatie te draaien kan je weer een nieuwe stack aanmaken via Portainer. Deze stack heeft niet alleen de Nextcloud applicatie nodig, maar ook een database. Standaard gebruikt Nextcloud SQLite, maar dat wordt niet aangeraden. Ik kies daarom nu voor een PostgreSQL database (ik weet niet of MariaDB, MySQL of een andere DB beter werkt, maar ik heb ervaring met Postgres en vind het altijd fijn werken en het is open source). Hieronder de YAML code met de stack.

```yaml
version: '3'  
  
volumes:  
  nextcloud-data:  
  nextcloud-db:  
  
  
networks:  
  frontend:  
    name: proxy  
    external: true  
        
  backend:  
  
  
services:  
  
  nextcloud-app:  
    image: nextcloud  
    restart: always  
    ports:  
      - 8080:80  
    volumes:  
      - nextcloud-data:/var/www/html  
    environment:  
      - POSTGRES_HOST=nextcloud-db  
      - POSTGRES_DB=${POSTGRES_DB}  
      - POSTGRES_USER=${POSTGRES_USER}  
      - POSTGRES_PASSWORD=${POSTGRES_PASSWORD}  
      - NEXTCLOUD_ADMIN_PASSWORD=${NEXTCLOUD_ADMIN_PASSWORD}  
      - NEXTCLOUD_ADMIN_USER=${NEXTCLOUD_ADMIN_USER}  
    networks:  
      - frontend  
      - backend  
  
  nextcloud-db:  
    image: postgres  
    restart: always  
    volumes:   
      - nextcloud-db:/var/lib/postgresql/data  
    environment:  
      - POSTGRES_USER=${POSTGRES_USER}  
      - POSTGRES_PASSWORD=${POSTGRES_PASSWORD}  
      - POSTGRES_DB=${POSTGRES_DB}  
    networks:  
      - backend
```

Deze stack is groter en ik zal weer kort toelichten wat er allemaal wordt gemaakt.

Als eerste worden er twee volumes gemaakt. [Volumes](https://docs.docker.com/storage/volumes/) maak je wanneer je de data van een container wilt behouden nadat de container is afgesloten of herstart. Het is belangrijk dat je dit voor Nextcloud doet, want anders ben je al je in de cloud opgeslagen bestanden kwijt wanneer de container stopt. Ook is er een volume voor de database en daarbij geldt hetzelfde.

Vervolgens wordt er gebruik gemaakt van twee [netwerken](https://docs.docker.com/network/), front-end en back-end. Voor de front-end specificeer je dat het een extern netwerk is met de naam proxy. Dat is het netwerk van de Nginx Proxy Manager stack! Hierdoor kunnen de applicaties die verbonden zijn aan dit netwerk met elkaar communiceren. Dat is nodig, want de Nginx Proxy gaat verkeer doorsturen naar de Nextcloud app. Het is ook goed om netwerken te scheiden, zodat applicaties alleen met elkaar kunnen communiceren als dat nodig is. Daarom is er een back-end netwerk gemaakt dat gebruikt wordt door de database en Nextcloud applicaties maar niet door de Nginx Proxy Manager!

Dan maken we nu de twee applicaties (docker noemt dat services). Als eerste de Nextcloud applicatie.Hierbij zijn een aantal zaken belangrijk:

-   onthouden welke poort je gebruikt (want deze is nodig voor de Proxy Host)
-   om de juiste volume te koppelen om zo je data te behouden
-   de juiste netwerken te kiezen (Nextcloud moet zowel met de database als de Nginx Proxy communiceren dus heeft deze applicatie zowel front-end als back-end netwerken nodig)
-   om Environment Variabelen in te stellen 

De environment variabelen zijn nodig om Nextcloud te configureren. Omdat er een Postgres DB wordt gebruikt zijn dat dus Postgres variabelen. Daarnaast maken we ook Nextcloud admin inlog gegevens aan ([hier](https://github.com/docker-library/docs/blob/master/nextcloud/README.md) is meer informatie over welke environment variablen je kan instellen in deze container).

Zoals je misschien opvalt zijn de waardes van de environment variabelen in dit format

`${POSTGRES_USER}`. Met deze syntax probeert Docker de waardes uit de environment variabelen te halen van de plek waar Docker compose wordt gedraaid. Dit doet Portainer voor je, maar dan moet je met het aanmaken van de stack deze wel definiëren. [Hier](https://www.portainer.io/blog/using-env-files-in-stacks-with-portainer) lees je hoe je dit kunt doen met een .env file, maar je kunt ze ook handmatig invoeren bij het aanmaken van de stack (onder de web editor).

Als laatste in deze stack maak je een Postgres DB. Deze moet verbonden zijn aan het back-end netwerk, aan de db volume. En je moet hier ook weer environment variabelen instellen. Voor meer informatie over de postgres container [klik hier](https://github.com/docker-library/docs/blob/master/postgres/README.md).

Nu beide stacks zijn aangemaakt, moeten er nog een aantal configuraties worden gedaan.

## Configuraties

### Nginx Proxy Configureren

Allereerst de configuratie van de Nginx Proxy. Via de Nginx Proxy Manager is het nodig om een Proxy Host in te stellen. Deze Proxy accepteert het inkomende verkeer en stuurt het door naar de Nextcloud applicatie. Hieronder staan twee screenshots met voorbeeld instellingen.

![](images/Group 1.png)

Belangrijk hierbij is dat je je aangemaakte domeinnaam toevoeg bij de Domain Names en het **lokale** IP adres van je Raspberry PI (het is daarom aan te raden dat je een statische IP voor je Raspberry Pi instelt). [Hier](https://www.tomshardware.com/how-to/static-ip-raspberry-pi) is uitleg hoe je dat kunt doen). En het laatste belangrijke punt is de Forward Port, dit moet de poort zijn waar de Nextcloud applicatie op draait. Deze poort is ingesteld in de Nextcloud stack hierboven en is als je dezelfde YAML hebt gebruikt de poort 8080.

Vervolgens stel je het SSL tabblad in met onderstaande gegevens.

![](images/Group%202.png)

Deze instellingen maken een nieuw SSL certificaat aan, gebruikmakend van [Lets Encrypt](https://letsencrypt.org/). Door dan SSL te forceren zal je altijd een versleutelde verbinding hebben wanneer je gegevens op gaat slaan in Nextcloud. Vergeet hier niet je eigen e-mailadres in te vullen.

### Router Configureren
Vervolgens moet je router nog zo ingesteld worden dat wanneer er iemand naar je router IP gaat, deze wordt doorverwezen naar je Nginx Proxy manager (en die stuurt het verkeer dan door naar je Nextcloud applicatie). Hiervoor moeten er twee poorten "geforward" worden, namelijk poort 80 (HTTP) en 443 (HTTPS). Hoe je dit doet is voor iedere router verschillend, maar dat ziet er ongeveer uit zoals onderstaande foto.

![](images/Pasted%20image%2020230510145327.png)

Voor het IP adres vul je het **lokale** Raspberry Pi adres in, dan wordt het verkeer naar je router op poort 80 en 443 doorgestuurd naar je Raspberry Pi.

### Nextcloud Configureren

Als laatste zijn er nog enkele wijzigingen nodig in de Nextcloud configuratie. Hiervoor moet je een PHP configuratie bestand aanpassen in je Nextcloud-data volume. De locatie van de Docker volume op je Raspberry Pi kan je vinden in Portainer door naar Volumes te navigeren en dan (als je dezelfde YAML configuraties heb gebruikt) de "nextcloud_nextcloud-data" te openen. Naast "Mount Path" staat dan de locatie waar het volume is opgeslagen. Hier kan je heen navigeren in je terminal op de Raspberry Pi. De configuratie file die aangepast moet worden is (wanneer je in de Mount Path folder zit):

> config/config.php

Open dit bestand met je favoriete editor (bijvoorbeeld vim) en dan moet je de volgende instellingen toevoegen:
```php
'overwrite.cli.url' => 'https://<jouw-domein>',
'overwritehost' => '<jouw-domein>',
'overwriteprotocol' => 'https',
```

Ook moet je (in deze config file) aan de Trusted Domains array je eigen domein toevoegen.

## Tot Slot

Wanneer je het configureren hebt voltooid kan je nu naar je domein navigeren om in te loggen met Nextcloud. Om vervolgens op je verschillende apparaten bij je Nextcloud bestanden te komen hoef je enkel nog Nextcloud te installeren.

Er zijn dan jammer genoeg een paar problemen... Namelijk dat Nextcloud op je mobiel geen two-way sync heeft, waardoor aanpassingen aan een bestand op je mobiel niet worden gesynchroniseerd naar je Nextcloud app. Ook is er geen snelle manier om een nieuwe notitie te maken met Obsidian op je mobiel, omdat er bijvoorbeeld geen Widgets van de Obsidian app zijn.

In een volgende post zal ik uitleggen hoe je deze twee problemen kunt oplossen.

Voor nu een fijne dag!

Mees

[^1]: <https://cabbagemees.nl/posts/2022-10-16-Flutter-app-deel-4-de-integration-test/>
[^2]: <https://docs.flutter.dev/cookbook/testing/widget/introduction>
[^3]: <https://api.flutter.dev/flutter/material/MaterialApp-class.html>
[^4]: <https://api.flutter.dev/flutter/material/Scaffold-class.html>
[^5]: <https://api.flutter.dev/flutter/material/AppBar-class.html>
[^6]: <https://api.flutter.dev/flutter/widgets/ListView-class.html>
[^7]: <https://api.flutter.dev/flutter/material/Card-class.html>